# Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy
from scipy.stats import pearsonr
import plotly.express as px  

from wordcloud import WordCloud, STOPWORDS

# Analysis 1

## Number of Players in the MLB Since 1948 by Last School Attended

In [ ]:
#Create dataframes for college playing information and list of schools
college_df = pd.read_csv("baseballdatabank-2022.2/contrib/CollegePlaying.csv")
schools_df = pd.read_csv("baseballdatabank-2022.2/contrib/Schools.csv")

#Sort college playing information by player, then year
college_df = college_df.sort_values(by=['playerID', 'yearID'], ascending=[False, False])

#Only keep data since from 1948 and after, convert year to string, group each player, and concatenate schoolIDs and years
college_df = college_df[college_df['yearID'] > 1947] 
college_df = college_df.astype({'yearID':'string'})
college_df = college_df.groupby(by="playerID").agg({'schoolID': ' '.join, 'yearID': ' '.join})

#Only want to look at the school that was most recently attended before entering the MLB (so if an athlete has transfered, the first institution attended is ignored)
college_df["schoolID"] = college_df["schoolID"].str.split(" ").str[0]
college_df["yearID"] = college_df["yearID"].str.split(" ").str[0]

#Add a player count column and group columns by the school (to get the number of MLB players from each school)
college_df['player_ct'] = 1
# college_df = college_df.groupby(by="schoolID").sum()
college_df = college_df.groupby(by="schoolID").agg({"player_ct":"sum"})


#Sort colleges by number of players and reset the index
college_df = college_df.sort_values(by=['player_ct'], ascending=[False])
college_df = college_df.reset_index()

#Drop city and country data from schools dataframe and merge with college playing dataframe
schools_df = schools_df.drop(columns=["city", "country"])
college_df = pd.merge(college_df, schools_df)

#Drop the school ID and rename "name_full" column to "school"
college_df = college_df.drop(columns=["schoolID"])
college_df.rename(columns = {'name_full': 'school'}, inplace = True)

#Keep only the top n schools by number of players in the MLB
n = 30
college_n_df = college_df.head(n)

#Create and plot bargraph
fig, ax = plt.subplots(figsize=(10, 6))
plot = sns.barplot(x=college_n_df.school, y=college_n_df['player_ct'])
plt.setp(plot.get_xticklabels(), rotation=90)
plt.title('Top ' + str(n) + ' Schools With the Most Number of Players Who Have Entered the MLB since 1948')
plt.xlabel('Schools')
plt.ylabel('# of Players')
plt.show()


#Create and plot wordcloud
college_n_df["tuples"] = list(zip(college_n_df.school, college_n_df.player_ct))
# college_n_df.iloc["tuples"] = list(zip(college_n_df.school, college_df.player_ct))

d = {}
for s, c in college_n_df["tuples"]:
    d[s] = c

wordcloud = WordCloud(prefer_horizontal=1).generate_from_frequencies(d)

plt.figure()
fig, ax = plt.subplots(figsize=(15, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## How many players are entering the MLB from schools in each state?

In [ ]:
#Drop school column to create a new data frame and sum the number of players who came out of a school in each given state
college_states_df = college_df.drop(columns=["school"])
college_states_df = college_states_df.groupby(by="state").sum()

#Reset index and rename columns
college_states_df = college_states_df.reset_index()
college_states_df.rename(columns = {'state': 'State', 'player_ct': 'Player Count'}, inplace = True)

#Create and plot bargraph
fig, ax = plt.subplots(figsize=(10, 6))
plot = sns.barplot(x=college_states_df.State, y=college_states_df['Player Count'])
plt.setp(plot.get_xticklabels(), rotation=90)
plt.title('Number of Players Entering the MLB from Schools in Each State since 1948')
plt.xlabel('States')
plt.ylabel('# of Players')
plt.show()

#Plot US heat map
fig = px.choropleth(college_states_df,  
                    locations="State",  
                    color="Player Count", 
                    hover_name="State", 
                    locationmode = 'USA-states')
fig.update_layout(
    title_text = 'What States are Home to the Schools Sending the Most Players to the MLB?', 
    geo_scope='usa',
)
fig.show()

# Analysis 2

## Obtain Data

In [ ]:
champs_df = pd.read_csv("baseballdatabank-2022.2/outside_data/ncaa_champs_history.csv")

#Removing covid year and any other year with no champs and make a copy 
champs_df = champs_df[champs_df.runnerup != "--"]
runnerup_df = champs_df.copy()

#Drop columns, so one dataframe has NCAA champions and the other has NCAA runnerups
champs_df = champs_df.drop(columns=["runnerup", "year"])
runnerup_df = runnerup_df.drop(columns=["champ", "year"])

#Change column names from champ/runnerup to school and create a title_count column which holds the number of respective titles by school
champs_df.rename(columns = {'champ': 'school'}, inplace = True)
runnerup_df.rename(columns = {'runnerup': 'school'}, inplace = True)
champs_df["title_count"] = 1
runnerup_df["title_count"] = 1
champs_df = champs_df.groupby(by="school").sum()
runnerup_df = runnerup_df.groupby(by="school").sum()

#Reset index and sort by number of titles
champs_df = champs_df.reset_index()
runnerup_df = runnerup_df.reset_index()
champs_df = champs_df.sort_values(by=["title_count"], ascending=[False])
runnerup_df = runnerup_df.sort_values(by=["title_count"], ascending=[False])

#Create a dataframe which stores the number of both champion and runnerup titles for each school (remove whitespace), sort by number of titles, reset index
combined_df = pd.concat([champs_df, runnerup_df])
combined_df = combined_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
combined_df["school"].str.strip()
combined_df = combined_df.groupby(by=["school"]).sum()
combined_df = combined_df.sort_values(["title_count"], ascending=[False])
combined_df = combined_df.reset_index()

## Champions (number of NCAA championship titles by school)

In [ ]:
#Create and plot barchart
fig, ax = plt.subplots(figsize=(10, 6))
plot = sns.barplot(x=champs_df["school"], y=champs_df['title_count'])
plt.setp(plot.get_xticklabels(), rotation=90)
plt.title('Number of NCAA Championship Titles by School')
plt.xlabel('Schools')
plt.ylabel('# of Titles')
plt.show()

#Create and plot wordcloud
champs_df["tuples"] = list(zip(champs_df["school"], champs_df["title_count"]))
d = {}
for s, c in champs_df["tuples"]:
    d[s] = c

wordcloud = WordCloud(prefer_horizontal=1).generate_from_frequencies(d)

plt.figure()
fig, ax = plt.subplots(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Runnerups (number of NCAA runnerup titles by school)

In [ ]:
#Create and plot bargraph
fig, ax = plt.subplots(figsize=(10, 6))
plot = sns.barplot(x=runnerup_df["school"], y=runnerup_df['title_count'])
plt.setp(plot.get_xticklabels(), rotation=90)
plt.title('Number of NCAA Runnerup Titles by School')
plt.xlabel('Schools')
plt.ylabel('# of Titles')
plt.show()


#Create and plot wordcloud
runnerup_df["tuples"] = list(zip(runnerup_df["school"], runnerup_df["title_count"]))
d = {}
for s, c in runnerup_df["tuples"]:
    d[s] = c

wordcloud = WordCloud(prefer_horizontal=1).generate_from_frequencies(d)

plt.figure()
fig, ax = plt.subplots(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Combined (number of NCAA championship and runnerup titles by school)

In [ ]:
#Create and plot bargraph
fig, ax = plt.subplots(figsize=(10, 6))
plot = sns.barplot(x=combined_df.school, y=combined_df['title_count'])
plt.setp(plot.get_xticklabels(), rotation=90)
plt.title('Number of NCAA Championship and Runnerup Titles by School')
plt.xlabel('Schools')
plt.ylabel('# of Titles')
plt.show()

#Create and plot wordcloud
combined_df["tuples"] = list(zip(combined_df["school"], combined_df["title_count"]))
d = {}
for s, c in combined_df["tuples"]:
    d[s] = c

wordcloud = WordCloud(prefer_horizontal=1).generate_from_frequencies(d)

plt.figure()
fig, ax = plt.subplots(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Correlation between titles and players entering the MLB

In [ ]:
#Remove states column
college_df = college_df.drop(columns=["state"])

### Champions

In [ ]:
#Drop tuples column
champs_df = champs_df.drop(columns=["tuples"])

#Merge championship title info with college players info
titles_pros_df = pd.merge(champs_df, college_df)

#Get correlation coefficient
corr, _ = pearsonr(titles_pros_df.title_count, titles_pros_df.player_ct)

#Plot titles vs players sent to mlb
sns.scatterplot(x=titles_pros_df.title_count, y=titles_pros_df.player_ct)
plt.title('Number of NCAA Championship Titles vs Number of Players Entering the MLB')
plt.xlabel('# of Titles')
plt.ylabel('# of Players')
plt.xlim([0, 15])
plt.text(8, 5, 'Pearsons Correlation: %.3f' % corr)
plt.show()

### Runnerups

In [ ]:
#Drop tuples column
runnerup_df = runnerup_df.drop(columns=["tuples"])

#Merge runnerup title info with college players info
runnerup_titles_pros_df = pd.merge(runnerup_df, college_df)

#Get correlation coefficient
corr, _ = pearsonr(runnerup_titles_pros_df.title_count, runnerup_titles_pros_df.player_ct)

#Plot titles vs players sent to mlb
sns.scatterplot(x=runnerup_titles_pros_df.title_count, y=runnerup_titles_pros_df.player_ct)
plt.title('Number of NCAA Runnerup Titles vs Number of Players Entering the MLB')
plt.xlabel('# of Titles')
plt.ylabel('# of Players')
plt.xlim([0, 15])
plt.text(8, 5, 'Pearsons Correlation: %.3f' % corr)
plt.show()

### Combined champions and runnerups

In [ ]:
#Drop tuples column
combined_df = combined_df.drop(columns=["tuples"])

#Merge combined titles info with college players info
combined_titles_pros_df = pd.merge(combined_df, college_df)

#Get correlation coefficient
corr, _ = pearsonr(combined_titles_pros_df.title_count, combined_titles_pros_df.player_ct)

#Plot titles vs players sent to mlb
sns.scatterplot(x=combined_titles_pros_df.title_count, y=combined_titles_pros_df.player_ct)
plt.title('Number of NCAA Championship and Runnerup Titles vs Number of Players Entering the MLB')
plt.xlabel('# of Titles')
plt.ylabel('# of Players')
plt.xlim([0, 15])
plt.text(8, 5, 'Pearsons Correlation: %.3f' % corr)
plt.show()

# Analysis 3

# Analysis 4